# FAKE REVIEW NODE EMBEDDING

In [382]:
# importing all necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import networkx as nx
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from node2vec import Node2Vec as n2v
sns.set()

%matplotlib inline

In [383]:
import os, sys
from copy import deepcopy
import numpy as np
import pandas as pd

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [384]:
print(f"Torch version: {th.__version__}.  CUDA version: {th.version.cuda}")

Torch version: 1.12.1+cu113.  CUDA version: 11.3


In [385]:
dgl_version = f"dgl-cu{th.version.cuda.replace('.', '')}==0.7.2"
dgl_version

'dgl-cu113==0.7.2'

In [386]:
# Follow instructions at https://www.dgl.ai/pages/start.html
!{sys.executable} -m pip install {dgl_version} -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html


In [387]:
os.environ['DGLBACKEND'] = 'pytorch'
import dgl
from dgl import function as fn

In [388]:
dgl.__version__

'0.7.2'

In [389]:
from dgl.dataloading.pytorch import NodeDataLoader
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [390]:
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import argparse

from dgl.sampling import node2vec_random_walk

In [391]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('all')
import re
pd.set_option('max_colwidth', 800)


#importing Libraies
import networkx as nx
import matplotlib.pyplot as plt
from numpy import genfromtxt
import collections
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [392]:
#loading dataset
df=pd.read_csv('/content/category 8.csv')
df.head()

,category,rating,label,text_
0,Clothing_Shoes_and_Jewelry_5,5.0,CG,Granddaughter loves it. Bought boxer briefs for my daughter and she loved them.
1,Clothing_Shoes_and_Jewelry_5,3.0,CG,I wish the strap was more of a wide width. I normally wear a size 10 and ordered a
2,Clothing_Shoes_and_Jewelry_5,5.0,CG,"Nice, thick material 10 yr old. The only problem is that it's a little too large for"
3,Clothing_Shoes_and_Jewelry_5,5.0,CG,They are easy to use and have the wide width. I will keep them for the summer and they
4,Clothing_Shoes_and_Jewelry_5,5.0,CG,Love them!! Great flippies plus they fit well!!Wife loved it.


In [393]:
#df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [394]:
df.shape

(3848, 4)

In [395]:
df['label'].value_counts()

CG    1924
OR    1924
Name: label, dtype: int64

In [396]:
# dropping unnecessary columns. 
df.drop(['category', 'rating'], axis = 1, inplace=True)
df.head()

,label,text_
0,CG,Granddaughter loves it. Bought boxer briefs for my daughter and she loved them.
1,CG,I wish the strap was more of a wide width. I normally wear a size 10 and ordered a
2,CG,"Nice, thick material 10 yr old. The only problem is that it's a little too large for"
3,CG,They are easy to use and have the wide width. I will keep them for the summer and they
4,CG,Love them!! Great flippies plus they fit well!!Wife loved it.


In [397]:
z = {'OR' : 1, 'CG' : 0}  # 1 for original reviews and 0 for computer generated reviews
df['label'] = df['label'].map(z)

In [398]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)
    
    return text

In [399]:
# applying that into function.
df["clean_text"] = df['text_'].apply(lambda x: clean_text(x))
df.head(10)

,label,text_,clean_text
0,0,Granddaughter loves it. Bought boxer briefs for my daughter and she loved them.,Granddaughter loves it Bought boxer briefs for my daughter and she loved them
1,0,I wish the strap was more of a wide width. I normally wear a size 10 and ordered a,I wish the strap was more of a wide width I normally wear a size and ordered a
2,0,"Nice, thick material 10 yr old. The only problem is that it's a little too large for",Nice thick material yr old The only problem is that it s a little too large for
3,0,They are easy to use and have the wide width. I will keep them for the summer and they,They are easy to use and have the wide width I will keep them for the summer and they
4,0,Love them!! Great flippies plus they fit well!!Wife loved it.,Love them Great flippies plus they fit well Wife loved it
5,0,"Love these, perfect size and style. The only problem is that they are very thin.",Love these perfect size and style The only problem is that they are very thin
6,0,"Uncomfortable to wear. Because of that, I bought the third pair.",Uncomfortable to wear Because of that I bought the third pair
7,0,"i really love this t-shirt, the fabric is thick and thick, and the material is durable",i really love this t shirt the fabric is thick and thick and the material is durable
8,0,"These are strong and not overly bulky, they feel comfortable and will keep my feet warm.",These are strong and not overly bulky they feel comfortable and will keep my feet warm
9,0,"Fits very comfortable, i bought a size up.Very comfortable. I love the style and",Fits very comfortable i bought a size up Very comfortable I love the style and


In [400]:
# dropping the uncleaned text column "text"
df = df.drop(['text_'], axis=1)

In [401]:
# renaming the column
df.rename(columns = {'clean_text':'text', 'label': 'Class'}, inplace = True)
df.head()

,Class,text
0,0,Granddaughter loves it Bought boxer briefs for my daughter and she loved them
1,0,I wish the strap was more of a wide width I normally wear a size and ordered a
2,0,Nice thick material yr old The only problem is that it s a little too large for
3,0,They are easy to use and have the wide width I will keep them for the summer and they
4,0,Love them Great flippies plus they fit well Wife loved it


In [402]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

{'using', 'mine', 'except', 'eight', 'third', 'amount', 'upon', 'mostly', 'n’t', 'yourself', 'give', '’d', 'everywhere', 'their', 'yourselves', 'per', 'him', 'thereupon', 'this', 'without', 'hence', 'just', 'very', 'somehow', 'our', 'whether', 'for', 'are', 'top', "'d", 'get', 'more', 'often', '‘d', 'which', 'ten', 'does', 'perhaps', 'whereby', 'along', 'behind', 'indeed', 'call', 'had', 'some', 'unless', 'although', 'almost', '’re', 'anyone', 'doing', 'nobody', "'s", 'five', 'name', 'few', 'rather', 'beyond', 'seeming', 'see', 'on', "n't", 'above', 'noone', 'her', 'who', 'hundred', 'two', 'why', 'sixty', 'both', 'have', 'one', 'ours', 'became', 'really', 'an', 'herself', 'among', 'side', 'there', 'across', "'ve", 'formerly', 'another', 'below', 'yours', 'less', 'whereupon', 'becoming', 'us', 'themselves', 'thence', "'re", 'not', 'once', 'since', 'full', 'beside', '’s', 'my', 'even', 'any', 'a', 'hers', 'how', 'please', 'your', '’m', 'where', 'three', "'ll", 'whereafter', 'his', 'serio

In [403]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [404]:
sw_nltk.extend(['th','stay'])
print(len(sw_nltk))

181


In [405]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
#stop_words = set(stopwords.words('english'))
ps=PorterStemmer()
wnl=WordNetLemmatizer()
def clean_lemma(review):
    review = review.lower()
    review = review.split()
    review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
    review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
    review = ' '.join(review)
    return review

In [406]:
# using lemmatization
df['text']=df['text'].apply(clean_lemma)
df.head()

,Class,text
0,0,granddaughter love bought boxer brief daughter loved
1,0,wish strap wide width normally wear size ordered
2,0,nice thick material yr old problem little large
3,0,easy use wide width summer
4,0,love great flippies plus fit wife loved


### FEATURE EXTRACTION

In [407]:
x = df['text']
y = df['Class']

In [408]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer()
corpii=cv.fit_transform(x) 

In [409]:
feature_names = cv.get_feature_names()
#feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [410]:
len(feature_names)

5883

In [411]:
u1 = pd.DataFrame(corpii.toarray(), columns=feature_names)
u1.head()

,aa,abd,ability,able,abrasion,absolute,absolutely,absolutley,absorb,absorbent,...,zigzag,zip,zipper,zippered,zipping,zombie,zoo,zqwxpctg,zumba,zumiez
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### FEATURE SELECTION

In [412]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 500)
X_kbest_features = chi2_features.fit_transform(u1, y)

In [413]:
chi_support = chi2_features.get_support()

In [414]:
chi_feature = u1.loc[:,chi_support].columns.tolist()

In [415]:
chi_feature

['actual',
 'actually',
 'add',
 'additional',
 'adidas',
 'adjust',
 'adjustable',
 'affordable',
 'age',
 'air',
 'allow',
 'allows',
 'amazing',
 'amazon',
 'area',
 'asian',
 'ask',
 'attach',
 'average',
 'away',
 'baby',
 'bag',
 'baggy',
 'ball',
 'bargain',
 'beat',
 'bed',
 'bend',
 'better',
 'bikini',
 'binding',
 'birthday',
 'bit',
 'blue',
 'body',
 'bonus',
 'boob',
 'bottle',
 'bought',
 'bounce',
 'bow',
 'box',
 'bra',
 'bracelet',
 'brand',
 'brim',
 'buckle',
 'bulge',
 'bulky',
 'bust',
 'buyer',
 'buying',
 'came',
 'cape',
 'care',
 'carhartt',
 'carrying',
 'case',
 'casual',
 'cat',
 'cause',
 'center',
 'certainly',
 'chain',
 'cheap',
 'cheaper',
 'check',
 'chest',
 'child',
 'choice',
 'clean',
 'cleaner',
 'clip',
 'clog',
 'cloth',
 'clothes',
 'clothing',
 'collection',
 'color',
 'colored',
 'colorful',
 'comfortable',
 'coming',
 'compared',
 'complete',
 'completely',
 'con',
 'consider',
 'constantly',
 'construction',
 'continue',
 'control',
 'corn

In [416]:
X_kbest_features

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.04453501],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [417]:
u3 = pd.DataFrame(X_kbest_features, columns = chi_feature)
u3.head()

,actual,actually,add,additional,adidas,adjust,adjustable,affordable,age,air,...,workout,worn,wrangler,wrap,wrinkle,wrong,xl,yes,zip,zipper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [418]:
#from scipy.spatial.distance import cdist
#s = cdist(u3, u3, 'euclid')
#s

In [419]:
#X1=pd.DataFrame(s)
#X1

In [420]:
# taking average
y = np.average(u3)
y

0.003688777426164953

In [421]:
adjmat_df=u3.copy()

In [422]:
adjmat_df[adjmat_df < y]=0
adjmat_df[adjmat_df >= y]=1

In [423]:
adjmat_df

,actual,actually,add,additional,adidas,adjust,adjustable,affordable,age,air,...,workout,worn,wrangler,wrap,wrinkle,wrong,xl,yes,zip,zipper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3843,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3845,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [424]:
q = adjmat_df.to_numpy()
q

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [425]:
# SOURCE : https://github.com/dmlc/dgl/issues/3364
src, dst = np.nonzero(X_kbest_features)

In [426]:
src

array([   0,    0,    0, ..., 3847, 3847, 3847])

In [427]:
dst

array([ 38, 103, 170, ..., 469, 471, 488])

In [428]:
g = dgl.graph((src, dst))

In [429]:
n_nodes = g.num_nodes()
n_nodes

3848

In [430]:
nodes = g.nodes()
nodes

tensor([   0,    1,    2,  ..., 3845, 3846, 3847])

In [431]:
n_nodes1 = g.number_of_nodes()
n_nodes1

3848

In [432]:
g.ndata['feat'] = torch.tensor(X_kbest_features)
g.ndata['feat'] 

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0445],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [433]:
n_train = int(n_nodes1 * 0.6)
n_val = int(n_nodes1 * 0.2)

In [434]:
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(train_mask)
train_mask

tensor([False, False, False,  ..., False, False, False])

In [435]:
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(val_mask)
val_mask

tensor([False, False, False,  ..., False, False, False])

In [436]:
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(test_mask)
test_mask

tensor([False, False, False,  ..., False, False, False])

In [437]:
train_mask[:n_train] = True

In [438]:
val_mask[n_train:n_train + n_val] = True

In [439]:
test_mask[n_train + n_val:] = True

In [440]:
g.ndata['train_mask'] = train_mask

In [441]:
g.ndata['val_mask'] = val_mask

In [442]:
g.ndata['test_mask'] = test_mask

In [443]:
s_array = df['Class'].to_numpy()
s_array

array([0, 0, 0, ..., 1, 0, 1])

In [444]:
t1 = torch.tensor(s_array)
t1

tensor([0, 0, 0,  ..., 1, 0, 1])

In [445]:
g.ndata["label"] =t1

In [446]:
label = g.ndata["label"]

In [447]:
label[train_mask]

tensor([0, 0, 0,  ..., 1, 1, 1])

In [448]:
label[val_mask]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [449]:
label[test_mask]

tensor([1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,

In [450]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [451]:
val_mask

tensor([False, False, False,  ..., False, False, False])

In [452]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [453]:
clf = LogisticRegression()
clf.fit(g.ndata['feat'][train_mask], label[train_mask])

#y_hat_val_lr = clf.predict_proba(X_kbest_features[val_mask,:])
y_val_lr = clf.predict(g.ndata['feat'][val_mask])

In [454]:
accuracy_score(label[val_mask], y_val_lr)

0.8855656697009102

In [455]:
clf_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
clf_balanced.fit(g.ndata['feat'][train_mask], label[train_mask])

y_val_lrb = clf_balanced.predict(g.ndata['feat'][val_mask])
accuracy_score(label[val_mask], y_val_lrb)

0.8855656697009102

In [456]:
rf = RandomForestClassifier(n_estimators=400)
rf.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = rf.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.7958387516254877

In [457]:
gb = GradientBoostingClassifier(n_estimators=300, learning_rate=1.0, max_depth=1, random_state=0)
gb.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = gb.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.7737321196358907

In [458]:
ab = AdaBoostClassifier(n_estimators=400, random_state=0)
ab.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = ab.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.7503250975292588

In [459]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=7, shuffle=True)

In [460]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=10000) 
scores = cross_val_score(log, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8583637716450216


In [461]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC()
scores = cross_val_score(lsvm, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8713589015151516


In [462]:
from sklearn.linear_model import PassiveAggressiveClassifier
pa=PassiveAggressiveClassifier(max_iter=1000, random_state=7, tol=1e-3)
scores = cross_val_score(pa, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8542065746753247


In [463]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rf, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8469426406926406


In [464]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=300)
scores = cross_val_score(ab, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8274377705627705


In [465]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=300)
scores = cross_val_score(gb, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8456452922077922


In [466]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, 
              gamma=0,  importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
             n_estimators=200, n_jobs=16,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, subsample=1,
              tree_method='exact', use_label_encoder=False
              )
scores = cross_val_score(xgbc, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.8547314664502164


In [467]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [468]:
node_features = g.ndata['feat'].float()
node_labels = g.ndata['label']
train_mask = g.ndata['train_mask']
valid_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [469]:
node_features

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0445],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [470]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [471]:
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    model.train()
    # forward propagation by using all nodes
    logits = model(g, node_features)
    # compute loss
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # compute validation accuracy
    acc = evaluate(model, g, node_features, node_labels, test_mask)
    print(acc)
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    #print(loss.item())

    # Save model if necessary.  Omitted in this example.

0.5356679636835279
0.5914396887159533
0.6679636835278858
0.7120622568093385
0.7626459143968871
0.7976653696498055
0.8261997405966277
0.8469520103761349
0.8702983138780804
0.8832684824902723
0.8949416342412452
0.8988326848249028
0.9053177691309987
0.9169909208819714
0.9221789883268483
0.9260700389105059
0.9273670557717251
0.9312581063553826
0.9325551232166018
0.9325551232166018
0.9351491569390402
0.940337224383917
0.940337224383917
0.9429312581063554
0.9416342412451362
0.9429312581063554
0.9416342412451362
0.9429312581063554
0.9442282749675746
0.9455252918287937
0.9442282749675746
0.9442282749675746
0.9429312581063554
0.9429312581063554
0.9416342412451362
0.940337224383917
0.9442282749675746
0.9455252918287937
0.9481193255512321
0.9481193255512321
0.9481193255512321
0.9468223086900129
0.9455252918287937
0.9455252918287937
0.9442282749675746
0.9429312581063554
0.9429312581063554
0.9429312581063554
0.9429312581063554
0.9416342412451362
